In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml

tf.enable_eager_execution()



data = fetch_openml(data_id=1590, as_frame=True)
X = pd.get_dummies(data.data)
y_true = (data.target == '>50K') * 1


print(X.head())


/home/jwrickman/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jwrickman/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jwrickman/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jwrickman/.local/lib/python3.6/site-packages/tensorflow/python/framework/dty

    age    fnlwgt  education-num  capital-gain  capital-loss  hours-per-week  \
0  25.0  226802.0            7.0           0.0           0.0            40.0   
1  38.0   89814.0            9.0           0.0           0.0            50.0   
2  28.0  336951.0           12.0           0.0           0.0            40.0   
3  44.0  160323.0           10.0        7688.0           0.0            40.0   
4  18.0  103497.0           10.0           0.0           0.0            30.0   

   workclass_Private  workclass_Self-emp-not-inc  workclass_Self-emp-inc  \
0                  1                           0                       0   
1                  1                           0                       0   
2                  0                           0                       0   
3                  1                           0                       0   
4                  0                           0                       0   

   workclass_Federal-gov  ...  native-country_Guatemala  \
0  

In [2]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier


classifier = DecisionTreeClassifier(min_samples_leaf=10, max_depth=4)
classifier.fit(X, y_true)
y_pred = classifier.predict(X)

In [25]:
from fairlearn.metrics import MetricFrame
from sklearn.metrics import accuracy_score

sex = data.data['sex']
gm = MetricFrame(metrics=accuracy_score, y_true=y_true, y_pred=y_pred, sensitive_features=sex)
print(gm.overall)
print(gm.by_group)


0.8443552680070431
sex
Female    0.925148
Male      0.804288
Name: accuracy_score, dtype: float64


In [4]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  df = pd.get_dummies(dataframe.data)
  labels = (dataframe.target == '>50K') * 1

  ds = tf.data.Dataset.from_tensors((np.array([df[col] for col in df.columns]).T, labels))
  print("here")
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

df = pd.get_dummies(data.data)

batch_size = 5
train_ds = df_to_dataset(data, batch_size=batch_size)

here


In [13]:


model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(105, )),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2)
])

In [14]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [19]:
df = pd.get_dummies(data.data)
y_train = (data.target == '>50K') * 1
b = np.zeros((y_train.size, y_train.max()+1))
b[np.arange(y_train.size),y_train] = 1

x_train = np.array([df[col] for col in df.columns]).T
x_train = (x_train - np.mean(x_train, axis=0)) / np.std(x_train, axis=0)
model.fit(x=x_train, y=b, epochs=2)

Epoch 1/2
48842/48842 [==============================] - 33s 668us/step - loss: 1.2522 - acc: 0.3404
Epoch 2/2
48842/48842 [==============================] - 31s 643us/step - loss: 1.0528 - acc: 0.3529


In [20]:
predictions = model.predict(x_train)


In [21]:
predictions = tf.argmax(predictions, axis=1)
print(predictions.shape)
gm = MetricFrame(metrics=accuracy_score, y_true=y_true, y_pred=predictions, sensitive_features=sex)

(48842,)


In [24]:
print(gm.overall)
print(gm.by_group)

0.8319069653167356
sex
Female    0.909523
Male      0.793415
Name: accuracy_score, dtype: float64
